In [ ]:
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

# The Birthday Paradox
### Andrew Weatherman
The birthday paradox asks for the probability that at least two people will share a birthdate (year excluded) from a sample of $n$ people. Superficially, the solution is counterintuitive; **from a random sample of 23 people, the probability that at least two people will share a birthdate is 50%.** That sounds silly! But...it's perfectly reasonable. Probabilities are a weird thing: Let's break down the Birthday Paradox.

In [ ]:
"""Import modules and define functions"""
import time
from calendar import month_abbr
import random
from collections import Counter
import matplotlib.pyplot as plt
from math import factorial, sqrt
from numpy import log, arange
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output

def find_match(dn):
    """A function that finds if there is a birthday match by generating dn birthdays once;
    used in combination with simulation() to loop over iterations"""
    match = 0
    # grab random months
    sim_month = random.choices(range(1,13), k=dn)
    # grab random days -- agreeing with month-day limits
    sim_day = [random.randint(1, 31) if any(months in sim_month for months in [1,3,5,7,8,10,12]) else random.randint(1,30) if any(months in sim_month for months in [4,6,9,11]) else random.randint(1,28) for months in range(len(sim_month))]
    # join lists
    sim_cpu_bday = [f'{sim_month[i]}, {sim_day[i]}' for i in range(len(sim_month))]
    # check if match occurs
    if any(i >= 2 for i in Counter(sim_cpu_bday).values()):
        match += 1
    return match

def simulation(dn, sample, multi=True, plot=False):
    """Runs simulation of size sample over range(2, dn+1) dates or a single number dn"""
    # run simulation over multiple dates
    if multi:
        # define empty lists for plotting
        x = []
        freq = []
        # enumerate to capture dn with matches
        for dates, j in enumerate(range(2, dn+1), start=2):
            total_match = 0
            # loop over range of total dates to generate
            for i in range(1, sample+1):
                iter_match = find_match(dates)
                if iter_match == 1:
                    total_match += 1
            x = [dates, total_match]
            freq.append(x)
            if not plot:
                print(f"Running {i} simulations of {dates} generated birthdays, we found {total_match} simulations with a match! Good for {'{:.2%}'.format(total_match / sample)}.")
        if plot:
            return freq
    # run simulation with one date 
    else:
        total_match = 0
        for i in range(1, sample+1):
            iter_match = find_match(dn)
            if iter_match == 1:
                total_match += 1
        print(f"Running {sample} simulations of {dn} generated birthdays, we found {total_match} simulations with a match! Good for {'{:.2%}'.format(total_match / sample)}.")

def prob_curve(dn, sample):
    """Plots a probability curve for sample simulations over a range of generated dates"""
    # grab simulated values
    freq = simulation(dn, sample, plot=True)
    dates = [freq[i][0] for i in range(len(freq))]
    n = [(freq[i][1]/sample) for i in range(len(freq))]
    plt.plot(dates, n, color='green')
    plt.xlabel("Generated birthdays")
    plt.ylabel("Matches ratio")
    plt.show()

def general(rate, items):
    """A general formula to return choices needed to return probability rate from items"""
    num = round(sqrt(-2*(log(1-rate)))*sqrt(items),3)
    print(f"From {items} choices, {num} items need to be chosen to have a {'{:.2%}'.format(float(rate))} chance of at least one match.")


To first prove the paradox, we will randomly generate 23 birthdays and see if there is a match. For simplicity here, and throughout the rest of the module, we will assume there is no leap year.

In [ ]:
"""Single generation"""
# grab random months
month = random.choices(range(1,13), k=23)
# grab random days -- agreeing with month-day limits
day = [random.randint(1, 31) if any(months in month for months in [1,3,5,7,8,10,12]) else month + random.randint(1,30) if any(months in month for months in [4,6,9,11]) else month + random.randint(1,28) for months in range(len(month))]
# change month numbers to abbr.
month = [month_abbr[i] for i in month]
# join lists
cpu_bday = [f'{month[i]}, {day[i]}' for i in range(len(month))]
# check if there is a match
if any(i >= 2 for i in Counter(cpu_bday).values()):
    print('We randomly generated 23 birthdays and found a match!')
    time.sleep(2)
# if no match
else: 
    print('Drats -- no matches were found!')
    time.sleep(2)

In [ ]:
from turtle import clear


dates_button = widgets.Button(description='Show dates')
out = widgets.Output()

def show_dates(_):
    with out:
        clear_output()
        print(*cpu_bday, sep="\n")

dates_button.on_click(show_dates)
widgets.VBox([dates_button, out])

That's cool, but to further prove the paradox, we should run more simulations. Feel free to play around with the number of simulations to run below.

In [48]:
@widgets.interact(Simulations=(1000, 20000))
def single_sim_widget(Simulations):
    simulation(23, Simulations, multi=False)

interactive(children=(IntSlider(value=10500, description='Simulations', max=20000, min=1000), Output()), _dom_…

We can clearly see that as we approach an infinite number of simulations, the percentage of matches will converge to 50%. But what if we change the number of generated birthdays?

In [49]:
@widgets.interact(Dates=(1,50), Simulations=(1000, 20000))
def birthday_widget(Dates, Simulations):
    simulation(Dates, Simulations, multi=False)

interactive(children=(IntSlider(value=25, description='Dates', max=50, min=1), IntSlider(value=10500, descript…

## So what is happening under the hood?
Perhaps the most intuitive approach is to calculate the probability that no birthdates match. The probability of the first individual having a unique birthdate is, well, 100% -- or 365/365. With one fewer unique date avaliable, the probability of the second person having a distinct birthdate is 364/365. The pattern continues at  363/365, 362/365, etc. In short, the probability of no matches occurs when person B does not have the same birthdate as person A; person C does not have the same birthdate as person A *or* B; and on.

A great way to think about this is that we are actually comparing *far* more birthdates against one another than might seem. In a room of 23 people, you might be aware of the 22 other birthdates that are being compared against yours, but there are 231 *more* date comparisons going on behind the scenes. In fact, there are nearly **10 times as many** comparisons happening that do *not* involve you. If you approach the problem from this view, you will start to understand why the chance of a match is >50%.

Birthdates are (mostly) independent events, meaning that the birthdate of one individual does not affect the birthdate of another. It is important to note that dates in this context are not *entirely* independent (if A=B and B=C then A=C), but for the purposes of this module, we can assume independence with reasonable precision.

In statistics, the probability of independent events is reached by multiplying each individual probability. In other words, we need to multiply 365/365, 364/365, 363/365...and so on. A nifty algebraic trick is to take the factorial of the numerators. To create a generic formula, we need to introudce variable $n$ -- the number of dates to generate. With a little bit of algebraic magic, coupled with the logic above, we reach the expression: $365!/(365-n)!*365^n$

Using this, we can calculate the probability that in a random group of 23 people *no* matches occur. Subbing 23 for $n$, we return 49.3%.

Now remember, this is the probability that no matches occur. In statistics, the probability that at least one event occurs (in this case, date matches) is one minus the probability that no matches occur: 50.7%.

## Further Exploration
Now that we have the intuition out of the way, let's explore the paradox a bit further. We can extrapolate the probability curve for one match over some number $n$ of generated dates. Let's start by investigating the change in probability as we approach 23 generated dates over 10,000 simulations of $n$.

In [ ]:
prob_curve(23, 10000)

As $n$ remains low, the probability of a match grows exponentially, but as we approach larger values of $n$, the growth levels off. Let's re-run the same simulation but extend $n$ to 50.

In [ ]:
prob_curve(50, 10000)

As we approach $n$ = 50, the rate of growth decreases. If we were to extend $n$ to larger bounds, the plateau would be even more apparent. In mathematics, we call this plateau an "asymptote," a line that a curve approaches, but never touches, as it heads to infinity. After all, there is the *possibility* that no value $n$ results in a date match. As we generate more dates, that chance gets marginally smaller, but -- even if we were to generate 100,000 dates -- the chance is possible.

## Extensions
With the intuition beind the birthday paradox, we are able to generalize formulas to approximate any number of scenarios (match a specific date, near matches, number of days with at least one birthday, etc). For brevity, we will explore just one extension of the birthday paradox.

Perhaps the most applicable extension to the core birthday problem is finding the number of "choices" needed to reach a desired probability match rate $r$ from any number $n$ total items. Simply put, if I want, on average, a 50% match rate, how many items should I sample from some population size. The math behind this forumla leverages some tricky calculus, so if you're not comfortable with Taylor approxmation, just take the formula at face value.

$
\sqrt{-2ln(1-r)} * \sqrt{n}
$

Feel free to play around with the interactive calculator below.

In [52]:
@widgets.interact(Probability=(0.01, 0.99, 0.01), Sample=(5, 1000, 5))
def general_widget(Probability, Sample):
    general(Probability, Sample)

interactive(children=(FloatSlider(value=0.5, description='Probability', max=0.99, min=0.01, step=0.01), IntSli…

For the birthday paradox, we can model the number of choices needed to attain a probability rate $r$ of at least one match.

In [ ]:
num = [round(sqrt(-2*(log(1-i[1])))*sqrt(365),3) for i in enumerate(arange(0.01,1,0.01), start=1)]
p = [i[0] for i in enumerate(arange(0.01,1,0.01), start=1)]
plt.plot(p, num)
plt.xlabel("Probability of >= 1 match")
plt.ylabel("Number of dates needed")
plt.show()

## Mental Shortcut
The formula shown is impossible to calculate on the fly. If you want to find the number of choices in some sample size $n$ that will yield a 50% match rate, $\sqrt{n}$ will give you a rough estimate. Let's try it out with $n$ equal to {100,1000}.

In [ ]:
sq_rate = [sqrt(n[1]) for n in enumerate(range(100, 1001), start=1)]
sq_dn = [i[0] for i in enumerate(range(100, 1001), start=1)]
rate = [sqrt(-2*(log(1-0.5)))*sqrt(i[1]) for i in enumerate(range(100, 1001), start=1)]
dn = [i[0] for i in enumerate(range(100, 1001), start=1)]
plt.plot(dn, rate, label='Formula')
plt.plot(sq_dn, sq_rate, label='Shortcut')
plt.xlabel("Size of sample")
plt.ylabel("Choices needed for 50% match rate")
plt.legend()
plt.show()

It's not a perfect representation, in fact it consistently under-estimates the true value, but for a quick, formula-free calculation, it would certainly do the trick

## More
To explore more about the birthday paradox, [this Wikipedia page](https://en.wikipedia.org/wiki/Birthday_problem) has an excellent collection of extension formulas and cases.